In [1]:
import cv2
import numpy as np

import torch
import torchvision.transforms as transforms
# from PIL import Image, ImageDraw
#import PIL
from PIL import Image as PILImage
from PIL import ImageDraw as PILImageDraw

import matplotlib.pyplot as plt
import argparse
import os
from IPython.display import clear_output, display, Image
from facenet_pytorch import MTCNN, InceptionResnetV1

from model import *

from torch import nn, optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from face.models.inception_resnet_v1 import InceptionResnetV1

In [2]:
cuda_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Running on device: {cuda_device}')

Running on device: cuda:0


In [3]:
face_detect_model = MTCNN(keep_all=True, device=cuda_device)

In [4]:
#Emotion recognition
def load_trained_model(model_path):
    model = Face_Emotion_CNN()
    model.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage), strict=False)
    return model


model = load_trained_model('../models/emotion_m2pt.pt')
emotion_dict = {0: 'Neutral', 1: 'Happy', 2: 'Surprise', 3: 'Sad',
                    4: 'Angry', 5: 'Disgust', 6: 'Fear'}


val_transform = transforms.Compose([
        transforms.ToTensor()])

### Face Recognition

In [5]:
#The model requires two custom classes to load. 
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x
class normalize(nn.Module):
    def __init__(self):
        super(normalize, self).__init__()
        
    def forward(self, x):
        x = F.normalize(x, p=2, dim=1)
        return x

In [6]:
#Load our transfer trained model (with our custom classes for classmates)
model_ft = torch.load('face/models/face_recognition_transf.pt')
model_ft.to(cuda_device)
model_ft.eval()

Sequential(
  (0): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (1): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (2): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (5): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3,

In [7]:
#Now we load the data_transforms and class_names specified in the facerec_transfer_training notebook. 
data_transforms = {
    'train': transforms.Compose([
        #Note: Our images must be 160x160 for model (Training & Val). 
        transforms.Resize(size = (160,160)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(size = (160,160)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = 'face/data/train_val'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=8, 
                                             shuffle=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes
class_names

['Alice_Hua',
 'Brad_DesAulniers',
 'Darragh_Hanley',
 'Diana_Chacon',
 'Divesh_Kumar',
 'Eric_Lundy',
 'Jerico_Johns',
 'Josh_Jonte',
 'Kevin_Martin',
 'Piotr_Parkitny',
 'Sudhrity_Mondal']

In [8]:
def get_name(face_img):
    #face_img = face_img / 256
    face_img = PILImage.fromarray((face_img))
    val_transform = data_transforms['val']
    face_img = val_transform(face_img).unsqueeze(0)
    with torch.no_grad(): 
        model_ft.eval()
        face_img = face_img.to(cuda_device)
        output = model_ft(face_img)
        top_p, pred = output.topk(1,dim =1)
        pred = pred.cpu()
        name = class_names[int(pred.numpy())]
        return name

In [9]:
video_capture = cv2.VideoCapture(0)
frame_count = 0


# def get_name(face_img):
#     return 'josh'

# def get_emotion(face_img):
#     return 'happy'

# from PIL import Image

try:
    while(True):
        frame_count += 1
        
        clear_output(wait=True)
        _, frame = video_capture.read()
        faces, _ = face_detect_model.detect(frame)
        
        if faces is not None:
            
            
            for top, left, bottom, right  in faces:
                
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
                resize_frame = cv2.resize(gray[int(top):int(bottom), int(left):int(right)], (48, 48))

                X = resize_frame/256
                X = PILImage.fromarray((X))
                X = val_transform(X).unsqueeze(0)
                with torch.no_grad():
                    model.eval()
                    log_ps = model.cpu()(X)
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    pred = emotion_dict[int(top_class.numpy())] 
        
                #face_img = frame[0:300, 0:200]
                face_img = frame[int(top):int(bottom), int(left):int(right)]
                cv2.rectangle(frame, (top, left), (bottom, right), (0, 100, 0), 2)
                
                #Face name recognition
                rgb_face = face_img[:, :, ::-1]
                face_name = get_name(rgb_face)

                font = cv2.FONT_HERSHEY_DUPLEX
                x, y, w, h = int(top+2), int(right-50), int(left-2), int(bottom-26)

                scale = 0.045 # this value can be from 0 to 1 (0,1] to change the size of the text relative to the image
                fontScale = min(w,h)/(25/scale)

                cv2.putText(frame, face_name, ((int(top)+3), (int(right)-18)), font, fontScale, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(frame, pred, ((int(top)+3), (int(right)-4)), font, fontScale, (255, 255, 255), 1, cv2.LINE_AA)
               
   
        #ret, img = cv2.imencode('.jpg', frame)
        #img = Image(data=img)
        #display(img)
        
        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    video_capture.release()
  
        
    

KeyboardInterrupt: 